In [1]:
import math
import numpy as np
from qiskit import *
from qiskit.tools.visualization import plot_histogram, plot_state_city
from qiskit.providers.aer import QasmSimulator, StatevectorSimulator, UnitarySimulator
from qiskit import IBMQ
import matplotlib
%matplotlib inline

IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [2]:
def loadQuantumImage(path):
    from PIL import Image
    image= Image.open(path)
    (width, height) = image.size
    pixels = image.load()

    ssum = 0
    for i in range(width):
        for j in range(height):
            ssum += pixels[i,j]**2

    m = math.floor(math.log(height, 2))
    n = math.floor(math.log(width, 2))
    stateVector = np.zeros(2**(m+n))
    for i in range(width):
        for j in range(height):
            stateVector[i*height + j] = pixels[i,j]/math.sqrt(ssum)
            
    return stateVector, m, n

In [3]:
def exportQuantumImage(counts, shots, height, width):
    from PIL import Image

    r = math.floor(math.log(width, 2))
    im= Image.new("L", (width, height))
    pixels = im.load()

    maxAmplitude = 256
    medAmplitude = maxAmplitude/2
    med = shots/(height * width)
    for key in counts:
        i = int(key[0:r], 2)
        j = int(key[r:], 2)
        val = round((((counts[key] - med)/med)*medAmplitude) + medAmplitude)
        pixels[i,j] = (val)
        
    return im

In [4]:
def Increment(increment, quantumRegister, circuit):
    from qiskit.aqua.components.qfts import Standard as qft
    from qiskit.aqua.components.iqfts import Standard as iqft
    
    n = len(quantumRegister)
    
    qft(n).construct_circuit(qubits=quantumRegister,circuit=circuit)
    
    for j in range(n):
        circuit.u1((np.pi*increment)/(2**(n-1-j)), quantumRegister[j])
        
    iqft(n).construct_circuit(qubits=quantumRegister,circuit=circuit)
    

In [5]:
def Rotate(rowReg, columnReg, circuit):
    circuit.x(rowReg)
    circuit.x(columnReg)

In [6]:
def Transpose(rowReg, columnReg, circuit):
    m = rowReg.size
    n = columnReg.size
    
    if (n != m):
        raise "Can only transpose square images" 
    
    for i in range(n):
        circuit.swap(rowReg[i], columnReg[i])

In [ ]:
from qiskit.aqua.components.qfts import Standard as qft
stateVector, m, n = loadQuantumImage('Images\Documents\insurance1_10dg_64.bmp')

state = [complex(x) for x in stateVector]
indexes = range(m+n)

columnReg = QuantumRegister(n)
rowReg = QuantumRegister(m)
creg = ClassicalRegister(n+m)

circ = QuantumCircuit(rowReg, columnReg, creg)
circ.initialize(state, indexes)

qft(m).construct_circuit(qubits=rowReg,circuit=circ)
qft(n).construct_circuit(qubits=columnReg,circuit=circ)
#qft(n+m).construct_circuit(qubits=circ.qubits,circuit=circ)

Increment(2**(n-1), rowReg, circ)
Increment(2**(m-1), columnReg, circ)

circ.measure(indexes, indexes)

provider = IBMQ.get_provider(group='open')
localSimulator = Aer.get_backend('qasm_simulator')
backend = provider.get_backend('ibmq_qasm_simulator')

shots = 2**(m+n)*10 # 8190
result = execute(circ, localSimulator, shots=shots).result()
counts = result.get_counts(circ)
counts, m, n
            
exportedImg = exportQuantumImage(counts, shots, 2**m, 2**n)
exportedImg

In [20]:
#exportedImg.save("fain.jpg")
counts

{'011110100011': 1,
 '110111010000': 1,
 '110101011000': 1,
 '100100101101': 1,
 '101000000101': 1,
 '100000100011': 5,
 '111110111001': 1,
 '001101111010': 1,
 '011110001101': 2,
 '101001000001': 1,
 '110000010011': 1,
 '011111110101': 1,
 '100100111000': 3,
 '000111111110': 1,
 '010111011110': 1,
 '000011000111': 1,
 '001000111000': 1,
 '100010101100': 13,
 '010111000110': 1,
 '100010100000': 3,
 '100100110100': 2,
 '101010001101': 1,
 '110001011011': 1,
 '010001000101': 1,
 '100111100100': 2,
 '000100010011': 1,
 '010111011111': 1,
 '011010010100': 1,
 '011101001110': 3,
 '011011000011': 2,
 '100110001111': 1,
 '011010000000': 1,
 '110101100000': 1,
 '100000101100': 1,
 '100001100000': 3,
 '001010101001': 1,
 '100010101101': 1,
 '000010111100': 1,
 '011111011101': 1,
 '100011101111': 1,
 '100111110100': 1,
 '111101101111': 1,
 '000011101100': 1,
 '011111101010': 1,
 '100111111001': 1,
 '000010101001': 1,
 '110010100001': 1,
 '011000011010': 1,
 '011010011000': 1,
 '100110110110': 2,